In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train_query = pd.read_csv('./data/new/oppo_round1_train_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], sep='\t', usecols=['query_prediction'])
val_query = pd.read_csv('./data/new/oppo_round1_vali_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], sep='\t', usecols=['query_prediction'])
test_query = pd.read_csv('./data/new/oppo_round1_test_A_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag'], sep='\t', usecols=['query_prediction'])

df_query = pd.concat([train_query, val_query], axis=0)
df_query = pd.concat([df_query, test_query], axis=0)
df_query.reset_index(drop=True, inplace=True)
df_query.columns = ['query_prediction_origin']
df_query.shape

(2100000, 1)

In [3]:
df = pd.read_csv('./feature/df_preprocess_andtest.csv', encoding='utf-8')
df =  pd.concat([df, df_query], axis=1)

df.shape

(2100000, 16)

In [4]:
df.prefix = df.prefix.fillna('no_pre')

In [5]:
%%time
df['is_first_rate'] = df.apply(lambda x: 1 if x.dict_len != 0 and list(eval(x.query_prediction_origin).keys())[0] == x.title else 0, axis=1)
df['title_equal_prefix'] = df.apply(lambda x: 1 if x.prefix == x.title else 0, axis=1)
df['prefix_in_title'] = df.apply(lambda x: 1 if x.prefix in x.title else 0, axis=1)
df['title_startswith_prefix'] = df.apply(lambda x: 1 if x.title.startswith(x.prefix) else 0, axis=1)
df['title_endswith_prefix'] = df.apply(lambda x: 1 if x.title.endswith(x.prefix) else 0, axis=1)
df['rate_big5'] = df.apply(lambda x: 1 if x.title_rate >= 0.5 else 0, axis=1)
df['title_in_query'] = df.apply(lambda x: 1 if x.title in eval(x.query_prediction) else 0, axis=1)

df['title_rank_in_query'] = df.apply(lambda x: list(eval(x.query_prediction).keys()).index(x.title)+1 if x.title_in_query == 1 else 11, axis=1)
df['title_rank_in_query_origin'] = df.apply(lambda x: list(eval(x.query_prediction_origin).keys()).index(x.title)+1 if x.title_in_query == 1 else 11, axis=1)
df['rank_first_origin_rate'] = df.apply(lambda x: float(list(eval(x.query_prediction_origin).values())[0]) if x.dict_len != 0 else 0.6, axis=1)
df['rank_first_origin_rate_sub'] = df.apply(lambda x: x.title_rate - x.rank_first_origin_rate, axis=1)

Wall time: 13min 31s


In [6]:
df.shape
df.tail()

(2100000, 27)

,bayes_label,dict_len,is_max_rate,is_val,label,max_title_rate,prefix,query_prediction,tag,title,title_rate,title_rate_max_sub,title_tag,user_id,item_id,query_prediction_origin,is_first_rate,title_equal_prefix,prefix_in_title,title_startswith_prefix,title_endswith_prefix,rate_big5,title_in_query,title_rank_in_query,title_rank_in_query_origin,rank_first_origin_rate,rank_first_origin_rate_sub
2099995,0.0,10,0,-1,NaN,0.328,中通,"{'中通快递单号查询': '0.328', '中通快递': '0.187', '中通快递查询...",知道,中通快递单号查询那个数字开头最快,0.000,0.328,中通快递单号查询那个数字开头最快_知道,11cdf612623c1ac1073e36b32f83b084,99fa77afb3130eace45775f21d3ebef9,"{""中通快递电话"": ""0.019"", ""中通快递查询"": ""0.167"", ""中通快递查询...",0,0,1,1,0,0,0,11,11,0.019,-0.019
2099996,0.0,9,0,-1,NaN,0.134,亚美,"{'亚美科技': '0.134', '亚美科技车智汇怎么样': '0.100', '亚美尼亚...",百科,亚美,0.000,0.134,亚美_百科,b25b254cad6d5a69dbdbd115f5178bdc,c3f982ad256ca1c65f85d98db78daba5,"{""亚美科技"": ""0.134"", ""亚美车智汇"": ""0.060"", ""亚美车智汇是传销吗...",0,1,1,1,1,0,0,11,11,0.134,-0.134
2099997,0.0,10,0,-1,NaN,0.239,淋巴细胞,"{'淋巴细胞百分比偏高': '0.239', '淋巴细胞百分比偏低': '0.085', '...",健康,淋巴细胞高,0.065,0.174,淋巴细胞高_健康,d1bb946ee46d52bf6ada187f4297b8cb,f1f671998c31b4178460988cc0d82705,"{""淋巴细胞比率偏高"": ""0.025"", ""淋巴细胞比率高"": ""0.018"", ""淋巴细...",0,0,1,1,0,0,1,4,10,0.025,0.040
2099998,0.0,9,0,-1,NaN,0.036,苹果,"{'苹果x': '0.036', '苹果官网': '0.032', '苹果8': '0.02...",经验,苹果手机怎么关闭系统更新提示,0.000,0.036,苹果手机怎么关闭系统更新提示_经验,a3b18e4ddb68015ba6d276784eec8de5,7f21cd8bd3738f24e5fed4d1ff853bc0,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",0,0,1,1,0,0,0,11,11,0.017,-0.017
2099999,0.0,10,0,-1,NaN,0.057,地瓜,"{'地瓜叶的功效与作用': '0.057', '地瓜图片': '0.045', '地瓜叶子怎...",菜谱,地瓜饼的做法,0.018,0.039,地瓜饼的做法_菜谱,33c23a029e15fd69119fab48ef8f43b9,55734d907484533b107b14f052b4cd69,"{""地瓜图片"": ""0.045"", ""地瓜叶"": ""0.030"", ""地瓜叶子怎么做好吃"":...",0,0,1,1,0,0,1,6,10,0.045,-0.027


## his

In [2]:
df = pd.read_csv('./feature/df_feature2.csv', encoding='utf-8')

In [3]:
from sklearn.cross_validation import train_test_split

C:\Users\ZERO\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
his, train, y_his, y_train = train_test_split(df[df.is_val == 0], df[df.is_val == 0].label, test_size=0.33, random_state=2018)
his.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [5]:
train = pd.concat([train, df[df.is_val == 1]], axis=0)
train = pd.concat([train, df[df.is_val == -1]], axis=0)

In [6]:
his.shape
train.shape

(1340000, 27)

(760000, 27)

## ctr

In [7]:
items = ['prefix', 'title', 'tag']

for item in tqdm_notebook(items):
    temp = his.groupby(item, as_index = False)['label'].agg({item + '_click':'sum', item + '_ctr':'mean', item + '_count':'count'})
    train = pd.merge(train, temp, on=item, how='left')
    
for i in tqdm_notebook(range(len(items))):
    for j in range(i + 1, len(items)):
        item_g = [items[i], items[j]]
        temp = his.groupby(item_g, as_index=False)['label'].agg({'_'.join(item_g) + '_click': 'sum', '_'.join(item_g) + '_ctr': 'mean', '_'.join(item_g) + '_count':'count'})
        train = pd.merge(train, temp, on=item_g, how='left')

A Jupyter Widget

A Jupyter Widget

## 额外特征

In [8]:
# df = pd.read_csv('./feature/df_feature3_ctr.csv', encoding='utf-8')

train.shape

(760000, 45)

In [9]:
his['prefix_len'] = his.prefix.apply(len)
train['prefix_len'] = train.prefix.apply(len)

In [10]:
items = ['prefix_len','dict_len','is_max_rate','is_first_rate','title_equal_prefix','prefix_in_title','title_startswith_prefix','title_endswith_prefix','rate_big5','title_in_query','title_rank_in_query','title_rank_in_query_origin']

for item in tqdm_notebook(items):
    temp = his.groupby(item, as_index=False)['label'].agg({item + '_click': 'sum', item + '_ctr': 'mean', item + '_count':'count'})
    train = pd.merge(train, temp, on=item, how='left')

for i in tqdm_notebook(range(len(items))):
    item_g = [items[i], 'tag']
    temp = his.groupby(item_g, as_index=False)['label'].agg({'_'.join(item_g) + '_click': 'sum', '_'.join(item_g) + '_ctr': 'mean', '_'.join(item_g) + '_count':'count'})
    train = pd.merge(train, temp, on=item_g, how='left')

A Jupyter Widget

A Jupyter Widget

In [11]:
train.shape

(760000, 118)

In [12]:
train.head()

,bayes_label,dict_len,is_max_rate,is_val,label,max_title_rate,prefix,query_prediction,tag,title,title_rate,title_rate_max_sub,title_tag,user_id,item_id,query_prediction_origin,is_first_rate,title_equal_prefix,prefix_in_title,title_startswith_prefix,title_endswith_prefix,rate_big5,title_in_query,title_rank_in_query,title_rank_in_query_origin,rank_first_origin_rate,rank_first_origin_rate_sub,prefix_click,prefix_ctr,prefix_count,title_click,title_ctr,title_count,tag_click,tag_ctr,tag_count,prefix_title_click,prefix_title_ctr,prefix_title_count,prefix_tag_click,prefix_tag_ctr,prefix_tag_count,title_tag_click,title_tag_ctr,title_tag_count,prefix_len,prefix_len_click,prefix_len_ctr,prefix_len_count,dict_len_click,dict_len_ctr,dict_len_count,is_max_rate_click,is_max_rate_ctr,is_max_rate_count,is_first_rate_click,is_first_rate_ctr,is_first_rate_count,title_equal_prefix_click,title_equal_prefix_ctr,title_equal_prefix_count,prefix_in_title_click,prefix_in_title_ctr,prefix_in_title_count,title_startswith_prefix_click,title_startswith_prefix_ctr,title_startswith_prefix_count,title_endswith_prefix_click,title_endswith_prefix_ctr,title_endswith_prefix_count,rate_big5_click,rate_big5_ctr,rate_big5_count,title_in_query_click,title_in_query_ctr,title_in_query_count,title_rank_in_query_click,title_rank_in_query_ctr,title_rank_in_query_count,title_rank_in_query_origin_click,title_rank_in_query_origin_ctr,title_rank_in_query_origin_count,prefix_len_tag_click,prefix_len_tag_ctr,prefix_len_tag_count,dict_len_tag_click,dict_len_tag_ctr,dict_len_tag_count,is_max_rate_tag_click,is_max_rate_tag_ctr,is_max_rate_tag_count,is_first_rate_tag_click,is_first_rate_tag_ctr,is_first_rate_tag_count,title_equal_prefix_tag_click,title_equal_prefix_tag_ctr,title_equal_prefix_tag_count,prefix_in_title_tag_click,prefix_in_title_tag_ctr,prefix_in_title_tag_count,title_startswith_prefix_tag_click,title_startswith_prefix_tag_ctr,title_startswith_prefix_tag_count,title_endswith_prefix_tag_click,title_endswith_prefix_tag_ctr,title_endswith_prefix_tag_count,rate_big5_tag_click,rate_big5_tag_ctr,rate_big5_tag_count,title_in_query_tag_click,title_in_query_tag_ctr,title_in_query_tag_count,title_rank_in_query_tag_click,title_rank_in_query_tag_ctr,title_rank_in_query_tag_count,title_rank_in_query_origin_tag_click,title_rank_in_query_origin_tag_ctr,title_rank_in_query_origin_tag_count
0,0.333333,10,0,0,1.0,0.205,纹身贴,"{'纹身贴怎么洗掉': '0.205', '纹身贴怎么贴': '0.141', '纹身贴图片...",健康,去掉身上的纹身贴,0.000,0.205,去掉身上的纹身贴_健康,bd5a7362412f10ecf7e0a2e334af67f2,48e1641200b234c9f4ae5e3f4320a235,"{""纹身贴图片"": ""0.067"", ""纹身贴纸"": ""0.034"", ""纹身贴怎么用"": ...",0,0,1,0,1,0,0,11,11,0.067,-0.067,0.0,0.000000,3.0,0.0,0.000000,2.0,27903.0,0.297613,93756,0.0,0.000000,2.0,0.0,0.000000,2.0,0.0,0.000000,2.0,3,112097.0,0.369627,303271,364025.0,0.369072,986325,354655.0,0.326363,1086690,476362.0,0.366433,1299999,389066.0,0.359522,1082175,477893.0,0.385235,1240523,91764.0,0.253125,362524,135146.0,0.375708,359710,470663.0,0.362885,1297004,295865.0,0.309907,954690,295865.0,0.309907,954690,295865.0,0.309907,954690,7398.0,0.283949,26054.0,19735.0,0.286617,68855.0,18397.0,0.257258,71512,26301.0,0.290812,90440,26691.0,0.309156,86335,27718.0,0.297688,93111,5176.0,0.189625,27296,3003.0,0.159785,18794,25561.0,0.285668,89478,15640.0,0.239297,65358,15640.0,0.239297,65358.0,15640.0,0.239297,65358.0
1,0.345515,10,1,0,0.0,0.578,今日头,"{'今日头条': '0.578', '今日头条下载': '0.055', '今日头条新闻':...",应用,今日头条,0.578,0.000,今日头条_应用,8c8a4bc1ba2fb9c184f46f6d68c01d22,70e7a194b948c7bcc36faca214942a77,"{""今日头条怎么赚钱"": ""0.017"", ""今日头条下载安装"": ""0.041"", ""今日...",0,0,1,1,0,1,1,1,7,0.017,0.561,19.0,0.339286,56.0,183.0,0.360947,507.0,78357.0,0.353689,221542,12.0,0.342857,35.0,12.0,0.307692,39.0,67.0,0.352632,190.0,3,112097.0,0.369627,303271,364025.0,0.369072,986325,143933.0,0.568209,253310,476362.0,0.366433,1299999,389066.0,0.359522,1082175,477893.0,0.385235,1240523,406824.0,0.416198,977476,363442.0,0.370749,980290,27925.0,0.649479,42996,202723.0,0.526130,385310,14309

In [13]:
train.to_csv('./feature/train_feature4_ctr_extra(33).csv', index=None, encoding='utf-8')